In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from auction.operations import create_auction_app, setup_auction_app, place_bid, close_auction
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')
print(os.environ.get('ALGOD_URL'))

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

http://localhost:4001


In [3]:
creator = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
seller = Account.from_mnemonic(os.environ.get("SELLER_MN"))

print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

#asset_id = create_dummy_asset(creator, 1000000000000000000, 3, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 30
print(f"start_time", start_time)
reserve = 1_000_000  # 1 Algo
token_amount = 100
increment = 100_000  # 0.1 Algo

start_time 1640613491


In [13]:
close_auction(client, 635, 550, creator, creator)

app_global_state {b'T': 499, b'TBA': 0, b'C': b'0\n\xbcQ\xb5>`\xa8\x1f\xf8Y\xeaV\xb5\xfd\xb3\xa7q\x83\xed\xdb\x13\x1a\xa7\xd2,\x8f\xf0\xc6\xbcW\xd0', b'TSA': 0}


KeyError: b'TK_ID'

In [6]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)

# storeApp.create_app()
# storeAppID = storeApp.app_id
storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID

# bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))
# bidder1 = Account.from_mnemonic(os.environ.get("BUYER1_MN"))
# bidder2 = Account.from_mnemonic(os.environ.get("BUYER2_MN"))

# if storeApp.is_opted_in(seller.get_address()) == False:
#   storeApp.optin_app(seller)
# if storeApp.is_opted_in(bidder.get_address()) == False:
#   storeApp.optin_app(bidder)
# if storeApp.is_opted_in(bidder1.get_address()) == False:
#   storeApp.optin_app(bidder1)
# if storeApp.is_opted_in(bidder2.get_address()) == False:
#   storeApp.optin_app(bidder2)

close_auction(client, 557, storeAppID, creator)

print(f"storeAppID", storeAppID)
print(b"storeApp address", get_application_address(storeAppID))


TypeError: close_auction() missing 1 required positional argument: 'closer'

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [ ]:

print(f"seller address: {seller.get_address()}")
appID = create_auction_app(
    client=client,
    sender=creator,
    seller=seller.get_address(),
    token_id=asset_id,
    token_amount=token_amount,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment,
    store_app_address=get_application_address(storeAppID)
)

#appID = 527
print(f"Auction App ID: {appID}")
print(f"Auction App Address: {get_application_address(appID)}")
print("Latest time:", get_app_global_state(client, appID))

Alice is setting up and funding token auction...

In [ ]:
setup_auction_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_holder=seller,
    token_id=asset_id,
    asset_amount=token_amount
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

In [ ]:
bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))
bidder1 = Account.from_mnemonic(os.environ.get("BUYER1_MN"))
bidder2 = Account.from_mnemonic(os.environ.get("BUYER2_MN"))

print(f"Bidder Address: {bidder.get_address()}")
print(f"Bidder1 Address: {bidder1.get_address()}")
print(f"Bidder2 Address: {bidder2.get_address()}")

In [ ]:
# _, last_round_time = get_last_block_timestamp(client)
# print(f"start_time", start_time)
# print(f"last_round_time", last_round_time)
# if last_round_time < start_time + 5:
#     sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
bidAmount = reserve
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

In [ ]:
_, lastRoundTime = get_last_block_timestamp(client)
print(f"lastRoundTime", lastRoundTime)
place_bid(client=client, app_id=appID, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

In [ ]:
optin_to_asset(asset_id, bidder)

bidAmount = reserve + 300_000
place_bid(client=client, app_id=appID, bidder=bidder1, bid_amount=bidAmount)

optin_to_asset(asset_id, bidder1)

bidAmount = reserve + 500_000
place_bid(client=client, app_id=appID, bidder=bidder2, bid_amount=bidAmount)

optin_to_asset(asset_id, bidder2)

Alice is closing out the auction....

In [ ]:
# _, lastRoundTime = get_last_block_timestamp(client)
# if lastRoundTime < end_time + 5:
#     sleep(end_time + 5 - lastRoundTime)

close_auction(client, appID, storeAppID, creator, seller)

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)
print(f"storeAppID", storeAppID)
#assert actualAppBalances == expectedAppBalances

from utils import get_app_local_state
print(get_app_local_state(client, storeAppID, seller.get_address()))
print(get_app_local_state(client, storeAppID, bidder.get_address()))
print(get_app_local_state(client, storeAppID, bidder1.get_address()))
print(get_app_local_state(client, storeAppID, bidder2.get_address()))

In [ ]:
#bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]
bidder1NftBalance = get_balances(client, bidder1.get_address())[asset_id]
#bidder2NftBalance = get_balances(client, bidder2.get_address())[asset_id]

print("Carla's NFT balance:", bidder1NftBalance, " for token ID: ", asset_id)

assert bidder1NftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, 591)
print(storeAppState)
print(encoding.encode_address(storeAppState[b"C"]))